# Solving 3-SAT Problem with Hybrid Quantum-Classical solver 

**Author:** Xinghui Jia \& Siwei Tan

**Date:** 9/4/2024

Based on paper "[Janus-CT: A Hybrid Approach for 3-SAT Problems by Integrating Quantum Annealer with CDCL][1]"

[1]: https://ieeexplore.ieee.org/abstract/document/10071022/


A propositional satisfiability (SAT) problem is to find an assignment for each variable to satisfy a given Boolean formula. A typical SAT problem is composed of multiple clauses represented as a disjunction of Boolean variables. 3-SAT problem is a special case of the SAT problem that has no more than three variables in each clause. 

3-SAT problem is a fundamental problem in various applications, such as artificial intelligence, circuit analysis, and protein structure prediction.  Since the 3-SAT problem is an NP-complete problem, the time complexity of classical algorithms increases exponentially with the numbers of clauses and variables. In this notebook, we introduce Janus-SAT, a hybrid approach that integrates quantum annealing （QA） with the classical Conflict-Driven Clause Learning (CDCL) algorithm to enable end-to-end acceleration for solving 3-SAT problems. 


<div style="text-align:center;">
    <img src="./pictures/4-1.hyqsat.png"  width="50%" height="50%">
</div>

The figure above shows the workflow of Janus-CT. It features a cross-iterative process, where the CDCL algorithm leverages the interpretation results of QA to guide the search and sends new clauses to the QA hardware for acceleration. The frontend receives unsatisfied clauses from the decision step and finds hard clauses that need to be deployed to the QA hardware. We perform a breadth-first traversal to generate a clause queue, where the head clauses have higher priority to be accelerated by QA. We then embed the clauses following the order to the QA hardware under topology constraints. Our embedding scheme effectively eliminates the time-consuming routing and adjustment, which reduces the embedding time to linear complexity. To minimize noise overhead, we also adjust the coefficients in the QA objective function.

The frontend generates the embedding configuration with normalized coefficients and sends them to the QA hardware for acceleration. QA only needs to execute one sample rather than multiple samples as errors can be resolved during the next CDCL conflict resolving. The backend reads the outputs, e.g., energy, and variable state, to guide the CDCL exploration. In the backend, we estimate the satisfaction probability of the embedded clauses and adapt several feedback strategies. Similar to classic CDCL, Janus-CT follows propagation and conflict resolving steps. If all clauses are satisfied, Janus-CT outputs “satisfiable”, or it tries to resolve the conflict and returns to the decision step. In a word, Janus-CT algorithm takes advantage of both quantum computing and classical computing.

In [1]:
import sys
sys.path.append('..')

from janusq.hyqsat import solve_by_janusct, solve_by_minisat

In [2]:
# Configure the model
file_path = "cnf_examples/uf50-01.cnf"  # input cnf flie
cpu_lim = 0  # limit the cpu time (s). 0 means infinite
mem_lim = 0  # limit the memory 0 means infinite
strictp = True  # isStrict

### Using Janus-SAT API to Solve the Problem

We can use a QA simulator or real-world QA hardware to solve the SAT roblem. For example, here we use the simualtor.

In [4]:
# solve by Janus-SAT
solve_by_janusct(file_path, verb=False, cpu_lim=cpu_lim, mem_lim=mem_lim)

{'restarts': 1,
 'conflicts': 8,
 'conflict cost': 0.063,
 'decisions': 0,
 'propagations': 0,
 'conflict literals': 40,
 'actual CPU time': 7.26835,
 'annealing time': 0.0,
 'quantum count': 0,
 'quantum success number': 8,
 'quantum conflict number': 13,
 'quantum one time solve number': 0,
 'isSatisfiable': True,
 'isSat': True}

Note that when using the simulator, the solving time is estimated as (CDCL time + number of QA $\times$ 120 $\mu s$).

### Comparision to MiniSAT

We compare it to the MiniSAT solver.

In [5]:
# solve by minisat
solve_by_minisat(file_path, verb=False, cpu_lim=cpu_lim, mem_lim=mem_lim)

{'restarts': 1,
 'conflicts': 16,
 'conflict cost': 0.109,
 'decisions': 0,
 'propagations': 0,
 'conflict literals': 65,
 'actual CPU time': 0.052606,
 'annealing time': 0.0,
 'quantum count': 0,
 'quantum success number': 0,
 'quantum conflict number': 0,
 'quantum one time solve number': 0,
 'isSatisfiable': True,
 'isSat': True}